## Recommendation Systems.
There are three main types of recommendation systems:
* **Content-based filtering** - looks at the item and recommends similar items eg action movie(Jumanji) then the recommendation can be a similar action movie (Tomb Raider)
* **Collaborative filtering** - looks at who liked what, then suggests these items.
* **Hybrid Models** - combines both for more nuanced recommendations.(Amazon, Netflix & Google Ads)

We'll start by looking at `content-based filtering` using cosine similarity on the movielens dataset.

In [9]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
#load the movie and genres dataset

items = ["movie_id", "movie title", "release date", "video_release_date",
            "IMDb URL", "unknown", "action", 'adventure', "animation",
              "children's", "comedy", "crime", "documentary", "drama", "fantasy",
              "Film-Noir", "horror", "musical", "mystery", "romance", "Sci-Fi",
              "thriller", "war", "western"]

movies = pd.read_csv('./data/mlk/u.item', sep="|", names=items, encoding='latin-1')
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682 entries, 0 to 1681
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movie_id            1682 non-null   int64  
 1   movie title         1682 non-null   object 
 2   release date        1681 non-null   object 
 3   video_release_date  0 non-null      float64
 4   IMDb URL            1679 non-null   object 
 5   unknown             1682 non-null   int64  
 6   action              1682 non-null   int64  
 7   adventure           1682 non-null   int64  
 8   animation           1682 non-null   int64  
 9   children's          1682 non-null   int64  
 10  comedy              1682 non-null   int64  
 11  crime               1682 non-null   int64  
 12  documentary         1682 non-null   int64  
 13  drama               1682 non-null   int64  
 14  fantasy             1682 non-null   int64  
 15  Film-Noir           1682 non-null   int64  
 16  horror

We'll do a bit of cleaning by removing column with the most null entries.

In [11]:
movies = movies.drop(['video_release_date','IMDb URL'], axis=1)
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682 entries, 0 to 1681
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   movie_id      1682 non-null   int64 
 1   movie title   1682 non-null   object
 2   release date  1681 non-null   object
 3   unknown       1682 non-null   int64 
 4   action        1682 non-null   int64 
 5   adventure     1682 non-null   int64 
 6   animation     1682 non-null   int64 
 7   children's    1682 non-null   int64 
 8   comedy        1682 non-null   int64 
 9   crime         1682 non-null   int64 
 10  documentary   1682 non-null   int64 
 11  drama         1682 non-null   int64 
 12  fantasy       1682 non-null   int64 
 13  Film-Noir     1682 non-null   int64 
 14  horror        1682 non-null   int64 
 15  musical       1682 non-null   int64 
 16  mystery       1682 non-null   int64 
 17  romance       1682 non-null   int64 
 18  Sci-Fi        1682 non-null   int64 
 19  thrill

In [12]:
movies.head()

,movie_id,movie title,release date,unknown,action,adventure,animation,children's,comedy,crime,...,fantasy,Film-Noir,horror,musical,mystery,romance,Sci-Fi,thriller,war,western
0,1,Toy Story (1995),01-Jan-1995,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


The columns seem to be one-hot encoded so lets create vectors to represent each movie. Then use these vectors in the cosine similarity function to get recommendations.

In [13]:
genres = [ "unknown", "action", 'adventure', "animation",
              "children's", "comedy", "crime", "documentary", "drama", "fantasy",
              "Film-Noir", "horror", "musical", "mystery", "romance", "Sci-Fi",
              "thriller", "war", "western"]

genre_features = movies[genres]

genre_features.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682 entries, 0 to 1681
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   unknown      1682 non-null   int64
 1   action       1682 non-null   int64
 2   adventure    1682 non-null   int64
 3   animation    1682 non-null   int64
 4   children's   1682 non-null   int64
 5   comedy       1682 non-null   int64
 6   crime        1682 non-null   int64
 7   documentary  1682 non-null   int64
 8   drama        1682 non-null   int64
 9   fantasy      1682 non-null   int64
 10  Film-Noir    1682 non-null   int64
 11  horror       1682 non-null   int64
 12  musical      1682 non-null   int64
 13  mystery      1682 non-null   int64
 14  romance      1682 non-null   int64
 15  Sci-Fi       1682 non-null   int64
 16  thriller     1682 non-null   int64
 17  war          1682 non-null   int64
 18  western      1682 non-null   int64
dtypes: int64(19)
memory usage: 249.8 KB


## Cosine Similarity
Instead of using angles directly, cosine similarity gives us a score between -1 and 1.
* 1 = items are identical in terms of direction
* 0 = they are completely different
* -1 = opposite direction

We will use these values to compare movies based on ttheir genre vectors.

In [14]:
#get the matrix to use within our function.
cosine_matrix = cosine_similarity(genre_features)

#displaying similarity of first movie with others
cosine_matrix[0][:15]
# cosine_matrix

array([1.        , 0.        , 0.        , 0.33333333, 0.        ,
       0.        , 0.        , 0.66666667, 0.        , 0.        ,
       0.        , 0.        , 0.57735027, 0.        , 0.        ])

In [15]:
movies.head(10)

,movie_id,movie title,release date,unknown,action,adventure,animation,children's,comedy,crime,...,fantasy,Film-Noir,horror,musical,mystery,romance,Sci-Fi,thriller,war,western
0,1,Toy Story (1995),01-Jan-1995,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
5,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,Twelve Monkeys (1995),01-Jan-1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
7,8,Babe (1995),01-Jan-1995,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
8,9,Dead Man Walking (1995),01-Jan-1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,10,Richard III (1995),22-Jan-1996,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


Next, lets define a function that will take a movie title as an input, use the list from the cosine matrix to sort similar movies then return a list of top 5 movies as the output.

In [24]:
def movie_recommendation(movie_title, n=5):
    movie_index = movies[movies['movie title'] == movie_title].index[0]

    scores = list(enumerate(cosine_matrix[movie_index]))

    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)

    #return the scores ommitting the first movie and adding the 6th one to make five 
    return_scores = sorted_scores[1: n+1]
    #get movie indices and names
    movie_indices = [i[0] for i in return_scores]

    return movies['movie title'].iloc[movie_indices]



movie_recommendation("Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)")

8       Dead Man Walking (1995)
14    Mr. Holland's Opus (1995)
17    White Balloon, The (1995)
18        Antonia's Line (1995)
29         Belle de jour (1967)
Name: movie title, dtype: object

In [25]:
movie_recommendation('Dead Man Walking (1995)')

8       Dead Man Walking (1995)
14    Mr. Holland's Opus (1995)
17    White Balloon, The (1995)
18        Antonia's Line (1995)
29         Belle de jour (1967)
Name: movie title, dtype: object

In [17]:
movies.tail(12)

,movie_id,movie title,release date,unknown,action,adventure,animation,children's,comedy,crime,...,fantasy,Film-Noir,horror,musical,mystery,romance,Sci-Fi,thriller,war,western
1670,1671,"Further Gesture, A (1996)",20-Feb-1998,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1671,1672,Kika (1993),01-Jan-1993,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1672,1673,Mirage (1995),01-Jan-1995,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1673,1674,Mamma Roma (1962),01-Jan-1962,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1674,1675,"Sunchaser, The (1996)",25-Oct-1996,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1675,1676,"War at Home, The (1996)",01-Jan-1996,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1676,1677,Sweet Nothing (1995),20-Sep-1996,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1677,1678,Mat' i syn (1997),06-Feb-1998,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


Improve on the function to make sure it doesn't return the movie title parsed into it.

In [18]:
def improved_movie_recommendation(movie_title, n=5):
    movie_index = movies[movies['movie title'] == movie_title].index[0]

    scores = list(enumerate(cosine_matrix[movie_index]))

    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)

    #make a list without the parsed in movie_title using the movie_id
    return_scores = []
    for score in sorted_scores:
        index = score[0]
        if index != movie_index:
            return_scores.append(score)
    #to break out of this loop
        if len(return_scores) == n:
            break

    movie_indices = [i[0] for i in return_scores]

    return movies['movie title'].iloc[movie_indices]


movie_recommendation('Dead Man Walking (1995)')

5     Shanghai Triad (Yao a yao yao dao waipo qiao) ...
8                               Dead Man Walking (1995)
14                            Mr. Holland's Opus (1995)
17                            White Balloon, The (1995)
18                                Antonia's Line (1995)
29                                 Belle de jour (1967)
Name: movie title, dtype: object

In [26]:
# movie_recommendation('B. Monkey (1998)')
improved_movie_recommendation('Dead Man Walking (1995)')

5     Shanghai Triad (Yao a yao yao dao waipo qiao) ...
14                            Mr. Holland's Opus (1995)
17                            White Balloon, The (1995)
18                                Antonia's Line (1995)
29                                 Belle de jour (1967)
Name: movie title, dtype: object

In [20]:
improved_movie_recommendation('B. Monkey (1998)')

606                         Rebecca (1940)
885           Life Less Ordinary, A (1997)
32                        Desperado (1995)
67                        Crow, The (1994)
89     So I Married an Axe Murderer (1993)
Name: movie title, dtype: object

In [21]:
improved_movie_recommendation('Dead Man Walking (1995)')


5     Shanghai Triad (Yao a yao yao dao waipo qiao) ...
14                            Mr. Holland's Opus (1995)
17                            White Balloon, The (1995)
18                                Antonia's Line (1995)
29                                 Belle de jour (1967)
Name: movie title, dtype: object

In [22]:
movie_recommendation('Dead Man Walking (1995)')


5     Shanghai Triad (Yao a yao yao dao waipo qiao) ...
8                               Dead Man Walking (1995)
14                            Mr. Holland's Opus (1995)
17                            White Balloon, The (1995)
18                                Antonia's Line (1995)
29                                 Belle de jour (1967)
Name: movie title, dtype: object

### Collaborative Filtering
We have seen an example of content-based filtering, now as you might have guessed its not a reliable system, just because two items are similar doesn't mean that the user will like it. 

So comes in collaborative filtering where we'll be looking at user preferences and recommending based on that. 

Collaborative Filtering (CF) is currently the most widely used approach to build recommendation systems and uses the users’ behavior in the form of user-item ratings for predictions.